<a href="https://colab.research.google.com/github/maharshoaib786/Auto-Trading-Bot/blob/main/King_Queen_Grid_Bot_(v121_3_PowerLogic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# king_queen_grid_bot.py  (v121.3-PowerLogic)
"""
An advanced grid bot for MetaTrader 5 featuring a dynamic "King/Queen" side management strategy.
This version pre-calculates the entire grid at startup and maintains a buffer of multiple pending orders.

Core Strategy:
The bot begins by placing a standard hedge (one BUY and one SELL). It then immediately pre-calculates
an entire grid of potential trade prices for both sides and stores them in memory. It aims to
maintain a constant number of open limit orders (e.g., 3) on each side. When a limit order
is filled, the bot places the next one from its in-memory list.

NEW Power Logic:
A `LOT_POWER_MULTIPLIER` can be set in the .env file. This value multiplies the lot size
of EVERY trade the bot opens, allowing for easy scaling of the entire operation. For example,
a power of 2.0 will double all lot sizes.

King/Queen Dynamic Strategy:
This is the bot's primary advanced feature, designed to manage a grid that has developed
a strong directional bias.

• Trigger Condition:
    - When the number of open trades on one side of a grid (e.g., BUYs) reaches the
      `KING_QUEEN_TRIGGER_COUNT`, the special mode is activated.

• Role Assignment:
    - The side that triggered the count becomes the "King Side" (the runner).
    - The opposite side becomes the "Queen Side" (the reactive hedger).

• King Side (Runner) Behavior:
    - Continues to place new pending grid orders from its pre-calculated list as the market
      moves in its favor.

• Queen Side (Hedger) Behavior:
    - Ceases its own grid-building activity.
    - It will only open a new position IF there are currently zero positions on the Queen side.
    - When it opens a trade, it's a single market order with special parameters:
        - Lot Size: It copies the lot size of the most recently opened King side position.
        - Stop Loss: Its SL is set to the Take Profit level of the most recent King side position.

• Grid Reset Condition:
    - Once all positions on the King side have been closed, the entire system resets.

Configuration is managed via a `.env` file.
"""
import os, sys, time, math, logging, traceback, datetime
import MetaTrader5 as mt5
from dotenv import load_dotenv

# Ensure UTF-8 encoding on Windows
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")

load_dotenv()


# --- Configuration ---
SYMBOL                          = os.getenv("SYMBOL", "XAUUSDc")
LOT_SMALL                       = float(os.getenv("LOT_SMALL", 0.01))
LOT_MAX                         = float(os.getenv("LOT_MAX", 20))
# NEW: Lot Power Multiplier
LOT_POWER_MULTIPLIER            = float(os.getenv("LOT_POWER_MULTIPLIER", 1.0))
GRID_LOT_SEQUENCE               = [0.01, 0.01, 0.02, 0.03, 0.03, 0.04, 0.06, 0.08, 0.10, 0.14, 0.18, 0.24, 0.31, 0.41, 0.55, 0.72, 0.95, 1.26, 1.67, 2.2, 2.91, 3.85, 5.09, 6.72, 8.89, 11.75, 15.53, 20.0]
GRID_STEP_PIPS                  = float(os.getenv("GRID_STEP_PIPS", 100))
DYNAMIC_STEP_PIPS               = float(os.getenv("DYNAMIC_STEP_PIPS", 100))
PROFIT_PIPS                     = float(os.getenv("PROFIT_PIPS", 350))
DYNAMIC_PROFIT_PIPS             = float(os.getenv("DYNAMIC_PROFIT_PIPS", 450))
DYNAMIC_POSITIONS_TRIGGER       = int(os.getenv("DYNAMIC_POSITIONS_TRIGGER", 27))
MAX_POSITIONS                   = int(os.getenv("MAX_POS", 60)) # Max total open positions across ALL grids
PROFIT_TARGET_AMT               = float(os.getenv("PROFIT_TARGET_AMT", 1000))
OPEN_DELAY                      = float(os.getenv("OPEN_DELAY", 2))
LOOP_MS                         = int(os.getenv("LOOP_MS", 100))
LOG_FILE                        = os.getenv("LOG_FILE", "king_queen_grid_bot.log")
LOGIN                           = int(os.getenv("MT5_LOGIN", 0))
PASSWORD                        = os.getenv("MT5_PASSWORD", "")
SERVER                          = os.getenv("MT5_SERVER", "")
PENDING_ORDER_EXPIRATION_MIN    = int(os.getenv("PENDING_ORDER_EXPIRATION_MIN", 0))
MAX_PENDING_ORDERS_AT_ONCE      = int(os.getenv("MAX_PENDING_ORDERS_AT_ONCE", 3))
LOG_BALANCE_INTERVAL            = int(os.getenv("LOG_BALANCE_INTERVAL", 100))
MAX_ACTIVE_GRIDS                = int(os.getenv("MAX_ACTIVE_GRIDS", 2)) # Limit for concurrent grids
INVALID_PRICE_RETRY_LIMIT       = int(os.getenv("INVALID_PRICE_RETRY_LIMIT", 10))
SLIPPAGE                        = int(os.getenv("SLIPPAGE", 30)) # Slippage for market orders
INVALID_PRICE_ADJUST_PIPS       = float(os.getenv("INVALID_PRICE_ADJUST_PIPS", 60.0)) # Pips to adjust price by on invalid price retry
ENABLE_TRADING_HOURS            = os.getenv("ENABLE_TRADING_HOURS", "False").lower() == "true"
TRADING_START_TIME_STR          = os.getenv("TRADING_START_TIME", "00:00") # Expected in HH:MM format, UTC
TRADING_END_TIME_STR            = os.getenv("TRADING_END_TIME", "23:59")   # Expected in HH:MM format, UTC
MAX_LOSS_AMT                    = float(os.getenv("MAX_LOSS_AMT", "0")) # Max allowable loss from initial_equity for current cycle. 0 or less disables.
KING_QUEEN_TRIGGER_COUNT        = int(os.getenv("KING_QUEEN_TRIGGER_COUNT", 10)) # Number of trades on one side to trigger King/Queen rename.

# --- Action Comment Strings & Search Keywords ---
ACTION_INITIAL_HEDGE_BUY = "initial_hedge_buy"
ACTION_INITIAL_HEDGE_SELL = "initial_hedge_sell"
ACTION_RE_HEDGE_BUY = "re_hedge_buy"
ACTION_RE_HEDGE_SELL = "re_hedge_sell"
ACTION_GRID_LIMIT_BUY = "gridbuy"
ACTION_GRID_LIMIT_SELL = "gridsell"
ACTION_CAPPED_BUY_TAG = "CappedBuy"
ACTION_CAPPED_SELL_TAG = "CappedSell"
ACTION_DYNAMIC_HEDGE_BUY_TAG = "DynamicHedgeBuy"
ACTION_DYNAMIC_HEDGE_SELL_TAG = "DynamicHedgeSell"
ACTION_SUFFIX_PROFIT_TARGET_CLOSE = "profit_target_close"
ACTION_SUFFIX_MAX_LOSS_CLOSE = "max_loss_limit_close"
SYSTEM_COMMENT_GRID_NAME = "System"
SEARCH_KEYWORD_CAPPED_GENERIC = "Capped"
SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC = "DynamicHedge"
SEARCH_KEYWORD_CAPPED_BUY_SPECIFIC = ACTION_CAPPED_BUY_TAG
SEARCH_KEYWORD_CAPPED_SELL_SPECIFIC = ACTION_CAPPED_SELL_TAG
SEARCH_KEYWORD_GRIDBUY_SANE = "gridbuy"
SEARCH_KEYWORD_GRIDSELL_SANE = "gridsell"
SEARCH_KEYWORD_MARKET_FALLBACK_SUBSTRING = "market_fallback"
SEARCH_KEYWORD_KINGSIDE = "KingSide"
SEARCH_KEYWORD_QUEENSIDE = "QueenSide"

# --- Logging setup ---
log = logging.getLogger("king_queen_grid_bot")
log.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
if not log.handlers:
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    file_handler = logging.FileHandler(LOG_FILE, mode="w", encoding="utf-8")
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    log.addHandler(file_handler)
log.propagate = False

# --- Parsed Configuration & Global State for Trading Hours ---
TRADING_START_TIME_OBJ: datetime.time | None = None
TRADING_END_TIME_OBJ: datetime.time | None = None
if ENABLE_TRADING_HOURS:
    try:
        TRADING_START_TIME_OBJ = datetime.datetime.strptime(TRADING_START_TIME_STR, "%H:%M").time()
        TRADING_END_TIME_OBJ = datetime.datetime.strptime(TRADING_END_TIME_STR, "%H:%M").time()
    except (ValueError, NameError):
        log.error(f"Invalid TRADING_START_TIME ('{TRADING_START_TIME_STR}') or TRADING_END_TIME ('{TRADING_END_TIME_STR}') format. Please use HH:MM. Trading hours feature disabled.")
        ENABLE_TRADING_HOURS = False

# --- State variables ---
initial_equity: float       = 0.0
grid_states = {}
base_magic_number = 12345
next_magic_number = 12346
loop_counter: int           = 0
connection_status_ok = True

# --- Connection Handling ---
def check_connection() -> bool:
    """Verifies and restores the MT5 connection."""
    global connection_status_ok
    terminal_info = mt5.terminal_info()
    if terminal_info is None or not hasattr(terminal_info, 'connected') or not terminal_info.connected:
        if connection_status_ok:
            log.warning("Connection to MT5 terminal lost. Attempting to reconnect...")
            connection_status_ok = False
        mt5.shutdown()
        time.sleep(5)
        if not mt5.initialize(login=LOGIN, password=PASSWORD, server=SERVER, timeout=10000):
            log.error(f"Failed to re-initialize MT5 connection, error code = {mt5.last_error()}")
            return False
        if mt5.account_info() is None:
            log.error(f"Re-initialization succeeded, but failed to get account info. Error: {mt5.last_error()}")
            return False
        log.info("✅ Successfully reconnected to MT5 terminal.")
        connection_status_ok = True
        return True
    if not connection_status_ok:
        log.info("Connection to MT5 terminal has been restored.")
        connection_status_ok = True
    return True

# --- Trading Hours Utility ---
def is_trading_session_active() -> bool:
    """Checks if the current time is within the configured trading hours."""
    if not ENABLE_TRADING_HOURS or TRADING_START_TIME_OBJ is None or TRADING_END_TIME_OBJ is None:
        return True
    terminal_info_val = mt5.terminal_info()
    if terminal_info_val is None or not hasattr(terminal_info_val, 'time'):
        if loop_counter % (LOG_BALANCE_INTERVAL * 5) == 1:
            log.warning("is_trading_session_active: Could not get terminal info or server time. Assuming trading is active.")
        return True
    server_datetime_utc = datetime.datetime.fromtimestamp(terminal_info_val.time, tz=datetime.timezone.utc)
    current_time_utc = server_datetime_utc.time()
    if TRADING_START_TIME_OBJ <= TRADING_END_TIME_OBJ:
        return TRADING_START_TIME_OBJ <= current_time_utc < TRADING_END_TIME_OBJ
    else: # Overnight case
        return current_time_utc >= TRADING_START_TIME_OBJ or current_time_utc < TRADING_END_TIME_OBJ

# --- Utilities ---
def mt5_login():
    """Initializes MT5 connection and logs in."""
    global initial_equity
    log.info("Attempting to initialize MetaTrader 5...")
    if not mt5.initialize(login=LOGIN, password=PASSWORD, server=SERVER, timeout=10000):
        log.error(f"MT5 initialize() failed, error code = {mt5.last_error()}"); mt5.shutdown(); sys.exit(1)
    log.info(f"MetaTrader 5 initialized successfully. Terminal version: {mt5.version()}")
    account_info = mt5.account_info()
    if account_info is None:
        log.error(f"Failed to get account info, error code = {mt5.last_error()}"); mt5.shutdown(); sys.exit(1)
    log.info(f"Logged in to account: {account_info.login}, Server: {account_info.server}, Balance: {account_info.balance:.2f} {account_info.currency}")
    if not mt5.symbol_select(SYMBOL, True):
        log.error(f"Symbol {SYMBOL} not found or not enabled in Market Watch, error code = {mt5.last_error()}"); mt5.shutdown(); sys.exit(1)
    symbol_info = mt5.symbol_info(SYMBOL)
    if symbol_info is None:
        log.error(f"Failed to get info for symbol {SYMBOL}, error code = {mt5.last_error()}"); mt5.shutdown(); sys.exit(1)
    log.info(f"Symbol {SYMBOL} selected. Description: {symbol_info.description}, Digits: {symbol_info.digits}")
    if initial_equity == 0.0:
        initial_equity = account_info.equity
        log.info(f"✅ Login successful. Initial Equity for profit tracking set to: {initial_equity:.2f} {account_info.currency}")
    else:
        log.info(f"✅ Re-login successful. Current Equity: {account_info.equity:.2f} {account_info.currency}. Profit tracking continues from {initial_equity:.2f}.")

def pip_val() -> float:
    """Determines the value of a single pip for the symbol."""
    info = mt5.symbol_info(SYMBOL)
    if not info:
        log.critical(f"CRITICAL: Could not get symbol info for {SYMBOL}. Cannot determine pip value."); raise ValueError("Symbol info not found.")
    if info.digits in [5, 4]: return 0.0001
    if info.digits in [3, 2]: return 0.01
    log.warning(f"Uncommon number of digits ({info.digits}) for {SYMBOL}. Using point value as pip value: {info.point}")
    return info.point

def tp_price(direction: str, entry_price: float, pips_to_use: float) -> float:
    """Calculates the Take Profit price level."""
    price_offset = pips_to_use * pip_val()
    s_info_digits = mt5.symbol_info(SYMBOL).digits
    return round(entry_price + price_offset, s_info_digits) if direction == "BUY" else round(entry_price - price_offset, s_info_digits)

def adjust_lot(lot: float) -> float:
    """Adjusts a lot size to conform to the symbol's volume rules."""
    info = mt5.symbol_info(SYMBOL)
    if not info:
        log.error(f"Could not get symbol info for {SYMBOL} in adjust_lot. Returning original lot.")
        return lot
    volume_step, volume_min, volume_max = info.volume_step, info.volume_min, info.volume_max
    if volume_step <= 0: volume_step = 0.01
    lot = round(lot / volume_step) * volume_step
    lot = max(volume_min, lot)
    lot = min(volume_max, lot, LOT_MAX)
    precision = abs(int(math.log10(volume_step))) if 0 < volume_step < 1 else 0
    return round(lot, precision)

def format_mt5_comment(grid_name: str, action: str) -> str:
    """Formats a comment string to be MT5 compliant."""
    sane_action = "".join(filter(str.isalnum, action)) or "trade"
    return f"Grid{grid_name}_{sane_action}"[:31]

# --- Trading Functions ---
def send_market_order(direction: str, lot: float, magic: int, grid_name: str, action_comment_str: str, stop_loss: float = 0.0, take_profit: float | None = None) -> mt5.OrderSendResult | None:
    """Sends a market order with retry logic."""
    mt5_comment = format_mt5_comment(grid_name, action_comment_str)
    adjusted_lot = adjust_lot(lot)
    if adjusted_lot < getattr(mt5.symbol_info(SYMBOL), "volume_min", 0.01):
        log.warning(f"Market Order: Adjusted lot {adjusted_lot} is below minimum for {direction} {lot} ({mt5_comment}). Order not sent.")
        return None
    price, calculated_tp = 0.0, 0.0
    for attempt in range(5):
        tick = mt5.symbol_info_tick(SYMBOL)
        if tick and tick.bid > 0 and tick.ask > 0:
            price = tick.ask if direction == "BUY" else tick.bid
            pips_to_use = DYNAMIC_PROFIT_PIPS if len(mt5.positions_get(symbol=SYMBOL, magic=magic) or []) + 1 >= DYNAMIC_POSITIONS_TRIGGER else PROFIT_PIPS
            calculated_tp = take_profit if take_profit is not None else tp_price(direction, price, pips_to_use)
            break
        log.warning(f"Market Order: Attempt {attempt+1}/5: No valid tick for {direction} ({mt5_comment}). Retrying...")
        time.sleep(0.2)
    else:
        log.error(f"Market Order: Failed to get valid tick for {direction} ({mt5_comment}). Cannot send.")
        return None

    request = {"action": mt5.TRADE_ACTION_DEAL, "symbol": SYMBOL, "volume": adjusted_lot, "type": mt5.ORDER_TYPE_BUY if direction == "BUY" else mt5.ORDER_TYPE_SELL, "price": price, "tp": calculated_tp, "sl": stop_loss, "deviation": SLIPPAGE, "magic": magic, "comment": mt5_comment, "type_time": mt5.ORDER_TIME_GTC}
    s_info = mt5.symbol_info(SYMBOL)
    if s_info and hasattr(s_info, 'filling_modes'):
        allowed = s_info.filling_modes
        if mt5.ORDER_FILLING_IOC in allowed: request["type_filling"] = mt5.ORDER_FILLING_IOC
        elif mt5.ORDER_FILLING_FOK in allowed: request["type_filling"] = mt5.ORDER_FILLING_FOK
        elif allowed: request["type_filling"] = allowed[0]

    log.info(f"Sending Market Order: {direction} {adjusted_lot} {SYMBOL} @ Market (Ref: {price:.5f}) TP: {calculated_tp:.5f} SL: {stop_loss:.5f} Magic: {magic} Comment: {mt5_comment}")
    result = mt5.order_send(request)

    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        log.info(f"🟢 Market Order: {direction} {result.volume:.2f} @ {result.price:.5f} successfully placed.")
        return result
    else:
        log.error(f"🔴 Market Order send failed for {direction} {adjusted_lot}. Retcode: {result.retcode if result else 'N/A'}, Comment: {result.comment if result else 'N/A'}, Error: {mt5.last_error()}")
        return None

def place_limit_order(direction: str, price_level: float, lot: float, magic: int, grid_name: str, action_comment_str: str) -> mt5.OrderSendResult | None:
    """Places a limit order."""
    mt5_comment = format_mt5_comment(grid_name, action_comment_str)
    adjusted_lot = adjust_lot(lot)
    if adjusted_lot < getattr(mt5.symbol_info(SYMBOL), "volume_min", 0.01):
        log.warning(f"Limit Order: Adjusted lot {adjusted_lot} is below minimum for {direction} at {price_level:.5f}. Order not placed.")
        return None

    pips_to_use = DYNAMIC_PROFIT_PIPS if len(mt5.positions_get(symbol=SYMBOL, magic=magic) or []) + 1 >= DYNAMIC_POSITIONS_TRIGGER else PROFIT_PIPS
    limit_order_tp = tp_price(direction, price_level, pips_to_use)
    order_type = mt5.ORDER_TYPE_BUY_LIMIT if direction == "BUY" else mt5.ORDER_TYPE_SELL_LIMIT

    request = {"action": mt5.TRADE_ACTION_PENDING, "symbol": SYMBOL, "volume": adjusted_lot, "type": order_type, "price": price_level, "tp": limit_order_tp, "sl": 0.0, "magic": magic, "comment": mt5_comment, "type_time": mt5.ORDER_TIME_GTC}
    if PENDING_ORDER_EXPIRATION_MIN > 0:
        request["type_time"] = mt5.ORDER_TIME_SPECIFIED
        request["expiration"] = int(time.time()) + PENDING_ORDER_EXPIRATION_MIN * 60

    log.info(f"Placing Limit Order: {direction} {adjusted_lot} {SYMBOL} @ {price_level:.5f} TP: {limit_order_tp:.5f} Magic: {magic} Comment: {mt5_comment}")
    result = mt5.order_send(request)

    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        log.info(f"🟢 Limit Order: {direction} {result.volume:.2f} @ {result.price:.5f} successfully placed.")
        return result
    else:
        log.error(f"🔴 Limit Order placement FAILED. Retcode: {result.retcode if result else 'N/A'}, Comment: {result.comment if result else 'N/A'}, Error: {mt5.last_error()}")
        return None

def cancel_pending_orders_by_side(direction: str, magic: int | None = None):
    """Cancels all pending limit orders for a given side and magic number."""
    log.info(f"Cancelling pending {direction} orders for magic {magic if magic is not None else 'ALL'}.")
    order_type_to_cancel = mt5.ORDER_TYPE_BUY_LIMIT if direction == "BUY" else mt5.ORDER_TYPE_SELL_LIMIT
    cancelled_count = 0
    for order in mt5.orders_get(symbol=SYMBOL) or []:
        if (magic is None or order.magic == magic) and order.type == order_type_to_cancel:
            if mt5.order_send({"action": mt5.TRADE_ACTION_REMOVE, "order": order.ticket}).retcode == mt5.TRADE_RETCODE_DONE:
                log.info(f"Cancelled pending {direction} order {order.ticket}.")
                cancelled_count += 1
            else:
                log.error(f"Failed to cancel pending {direction} order {order.ticket}. Error: {mt5.last_error()}")
    if cancelled_count == 0:
        log.info(f"No pending {direction} orders found to cancel for magic {magic if magic is not None else 'ALL'}.")

def sync_all_tps(direction: str, magic: int):
    """Syncs the TP of all positions on one side to match the TP of the newest position."""
    grid_state = grid_states.get(magic)
    if not grid_state or grid_state.get(f'capped_{direction.lower()}', False): return

    side_type = mt5.ORDER_TYPE_BUY if direction == "BUY" else mt5.ORDER_TYPE_SELL
    positions = [p for p in mt5.positions_get(symbol=SYMBOL, magic=magic) or [] if p.type == side_type and SEARCH_KEYWORD_CAPPED_GENERIC not in p.comment and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]
    if not positions: return

    positions.sort(key=lambda p: p.time_msc, reverse=True)
    newest_position = positions[0]
    if newest_position.tp == 0: return

    new_tp_level = newest_position.tp
    synced_count = 0
    for p in positions:
        if p.ticket == newest_position.ticket or math.isclose(p.tp, new_tp_level, abs_tol=pip_val()*0.1): continue
        log.info(f"TP Sync for {direction} (magic {magic}): Position {p.ticket} TP {p.tp:.5f} -> {new_tp_level:.5f}")
        modify_request = {"action": mt5.TRADE_ACTION_SLTP, "position": p.ticket, "tp": new_tp_level, "sl": p.sl}
        if mt5.order_send(modify_request).retcode == mt5.TRADE_RETCODE_DONE:
            synced_count += 1
    if synced_count > 0:
        log.info(f"✅ TP Sync Summary for {direction} (magic {magic}): {synced_count} positions updated.")

# --- Grid Management ---
def generate_grid_levels(start_price: float, direction: str) -> list[float]:
    """Pre-calculates a full list of grid price levels."""
    levels, current_price = [], start_price
    s_info_digits = mt5.symbol_info(SYMBOL).digits
    for _ in range(len(GRID_LOT_SEQUENCE)):
        price_offset = GRID_STEP_PIPS * pip_val()
        next_price = round(current_price - price_offset if direction == "BUY" else current_price + price_offset, s_info_digits)
        levels.append(next_price)
        current_price = next_price
    log.info(f"Generated {len(levels)} grid levels for {direction} side, starting from {start_price:.{s_info_digits}f}.")
    return levels

def hedge_if_empty():
    """Places the initial BUY and SELL hedge and generates the first grid."""
    global grid_states
    if grid_states or (mt5.positions_get(symbol=SYMBOL) or []): return
    if ENABLE_TRADING_HOURS and not is_trading_session_active():
        if loop_counter % (LOG_BALANCE_INTERVAL * 10) == 1:
            log.info("hedge_if_empty: Outside trading hours. Initial hedge not placed.")
        return

    log.info("🔄 No active grids. Initiating initial grid (Magic: %d)...", base_magic_number)
    buy_result = send_market_order("BUY", LOT_SMALL * LOT_POWER_MULTIPLIER, base_magic_number, "A", ACTION_INITIAL_HEDGE_BUY)
    sell_result = send_market_order("SELL", LOT_SMALL * LOT_POWER_MULTIPLIER, base_magic_number, "A", ACTION_INITIAL_HEDGE_SELL)

    if buy_result and sell_result:
        grid_states[base_magic_number] = {
            "name": "A", "buy_sequence_index": 0, "sell_sequence_index": 0,
            "prev_buy_count": 1, "prev_sell_count": 1, "capped_buy": False, "capped_sell": False,
            "king_queen_side": None,
            "buy_grid_levels": generate_grid_levels(buy_result.price, "BUY"),
            "sell_grid_levels": generate_grid_levels(sell_result.price, "SELL"),
            "buy_levels_placed": 0, "sell_levels_placed": 0,
        }
        log.info("Initial hedge placed successfully for Grid A (magic %d).", base_magic_number)
        time.sleep(OPEN_DELAY)
    else:
        log.error("Failed to place one or both initial market hedge orders.")

def handle_closed_hedge(magic: int):
    """Detects when positions close and manages the grid state accordingly."""
    global grid_states
    grid_state = grid_states.get(magic)
    if not grid_state: return

    positions_in_grid = [p for p in mt5.positions_get(symbol=SYMBOL, magic=magic) or []]

    # --- King/Queen Reset Logic ---
    king_queen_direction = grid_state.get('king_queen_side')
    if king_queen_direction:
        king_side_type = mt5.ORDER_TYPE_BUY if king_queen_direction == "BUY" else mt5.ORDER_TYPE_SELL
        if not any(p.type == king_side_type and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment for p in positions_in_grid):
            log.warning(f"👑 King side ({king_queen_direction}) for grid {grid_state['name']} has closed. Resetting grid.")
            cancel_pending_orders_by_side("BUY", magic); cancel_pending_orders_by_side("SELL", magic)
            for p in positions_in_grid:
                close_direction = mt5.ORDER_TYPE_SELL if p.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                tick = mt5.symbol_info_tick(SYMBOL)
                if not tick: continue
                close_price = tick.bid if p.type == mt5.ORDER_TYPE_BUY else tick.ask
                mt5.order_send({"action": mt5.TRADE_ACTION_DEAL, "symbol": SYMBOL, "volume": p.volume, "type": close_direction, "position": p.ticket, "price": close_price, "deviation": SLIPPAGE, "magic": magic})
            del grid_states[magic]
        return

    # --- Normal Mode Logic ---
    buy_count = sum(1 for p in positions_in_grid if p.type == mt5.ORDER_TYPE_BUY and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment)
    sell_count = sum(1 for p in positions_in_grid if p.type == mt5.ORDER_TYPE_SELL and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment)

    if buy_count < grid_state.get("prev_buy_count", buy_count + 1) and not grid_state.get('capped_buy'):
        log.info(f"🔔 BUY side of grid {grid_state['name']} (magic {magic}) appears to have closed.")
        cancel_pending_orders_by_side("BUY", magic)
        if buy_count == 0 and sell_count > 0 and is_trading_session_active():
            log.info("Re-hedging BUY side and regenerating levels.")
            buy_result = send_market_order("BUY", LOT_SMALL * LOT_POWER_MULTIPLIER, magic, grid_state['name'], ACTION_RE_HEDGE_BUY)
            if buy_result:
                grid_state.update({"buy_sequence_index": 0, "buy_grid_levels": generate_grid_levels(buy_result.price, "BUY"), "buy_levels_placed": 0})
                time.sleep(OPEN_DELAY)

    if sell_count < grid_state.get("prev_sell_count", sell_count + 1) and not grid_state.get('capped_sell'):
        log.info(f"🔔 SELL side of grid {grid_state['name']} (magic {magic}) appears to have closed.")
        cancel_pending_orders_by_side("SELL", magic)
        if sell_count == 0 and buy_count > 0 and is_trading_session_active():
            log.info("Re-hedging SELL side and regenerating levels.")
            sell_result = send_market_order("SELL", LOT_SMALL * LOT_POWER_MULTIPLIER, magic, grid_state['name'], ACTION_RE_HEDGE_SELL)
            if sell_result:
                grid_state.update({"sell_sequence_index": 0, "sell_grid_levels": generate_grid_levels(sell_result.price, "SELL"), "sell_levels_placed": 0})
                time.sleep(OPEN_DELAY)

    grid_state["prev_buy_count"] = buy_count
    grid_state["prev_sell_count"] = sell_count

    if not positions_in_grid:
        log.warning(f"Grid {grid_state['name']} (magic {magic}) is empty. Deactivating.")
        del grid_states[magic]

def manage_pending_orders(magic: int):
    """Maintains a buffer of pending limit orders from the in-memory grid lists."""
    grid_state = grid_states.get(magic)
    if not grid_state or len(mt5.positions_get(symbol=SYMBOL) or []) >= MAX_POSITIONS or not is_trading_session_active(): return

    pending_orders = mt5.orders_get(symbol=SYMBOL, magic=magic) or []

    # --- BUY SIDE ---
    if not grid_state.get('capped_buy') and grid_state.get('king_queen_side') != 'SELL':
        pending_buys = sum(1 for o in pending_orders if o.type == mt5.ORDER_TYPE_BUY_LIMIT)
        while pending_buys < MAX_PENDING_ORDERS_AT_ONCE:
            placed_idx = grid_state.get("buy_levels_placed", 0)
            levels = grid_state.get("buy_grid_levels", [])
            if placed_idx >= len(levels): break

            lot_idx = grid_state.get("buy_sequence_index", 0) + placed_idx
            raw_lot = (GRID_LOT_SEQUENCE[lot_idx] if lot_idx < len(GRID_LOT_SEQUENCE) else GRID_LOT_SEQUENCE[-1]) * LOT_POWER_MULTIPLIER
            action = f"{SEARCH_KEYWORD_KINGSIDE}_{ACTION_GRID_LIMIT_BUY}" if grid_state.get('king_queen_side') == 'BUY' else ACTION_GRID_LIMIT_BUY

            if place_limit_order("BUY", levels[placed_idx], raw_lot, magic, grid_state['name'], action):
                grid_state["buy_levels_placed"] += 1; pending_buys += 1; time.sleep(0.1)
            else:
                break

    # --- SELL SIDE ---
    if not grid_state.get('capped_sell') and grid_state.get('king_queen_side') != 'BUY':
        pending_sells = sum(1 for o in pending_orders if o.type == mt5.ORDER_TYPE_SELL_LIMIT)
        while pending_sells < MAX_PENDING_ORDERS_AT_ONCE:
            placed_idx = grid_state.get("sell_levels_placed", 0)
            levels = grid_state.get("sell_grid_levels", [])
            if placed_idx >= len(levels): break

            lot_idx = grid_state.get("sell_sequence_index", 0) + placed_idx
            raw_lot = (GRID_LOT_SEQUENCE[lot_idx] if lot_idx < len(GRID_LOT_SEQUENCE) else GRID_LOT_SEQUENCE[-1]) * LOT_POWER_MULTIPLIER
            action = f"{SEARCH_KEYWORD_KINGSIDE}_{ACTION_GRID_LIMIT_SELL}" if grid_state.get('king_queen_side') == 'SELL' else ACTION_GRID_LIMIT_SELL

            if place_limit_order("SELL", levels[placed_idx], raw_lot, magic, grid_state['name'], action):
                grid_state["sell_levels_placed"] += 1; pending_sells += 1; time.sleep(0.1)
            else:
                break

def manage_king_queen_sides(magic: int):
    """Manages the Queen side of a K/Q grid."""
    grid_state = grid_states.get(magic)
    if not grid_state or not grid_state.get('king_queen_side') or not is_trading_session_active(): return

    king_side_str = grid_state['king_queen_side']
    queen_side_str = "SELL" if king_side_str == "BUY" else "BUY"
    queen_side_type = mt5.ORDER_TYPE_BUY if queen_side_str == "BUY" else mt5.ORDER_TYPE_SELL

    positions = mt5.positions_get(symbol=SYMBOL, magic=magic) or []
    if any(p.type == queen_side_type for p in positions): return

    log.info(f"Queen side ({queen_side_str}) for magic {magic} is empty. Opening new Queen position.")
    king_side_type = mt5.ORDER_TYPE_BUY if king_side_str == "BUY" else mt5.ORDER_TYPE_SELL
    king_positions = [p for p in positions if p.type == king_side_type and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]

    if king_positions:
        king_positions.sort(key=lambda p: p.time_msc, reverse=True)
        last_king_pos = king_positions[0]
        if last_king_pos.tp > 0:
            action = f"{SEARCH_KEYWORD_QUEENSIDE}_{queen_side_str.lower()}"
            send_market_order(queen_side_str, last_king_pos.volume, magic, grid_state['name'], action, stop_loss=last_king_pos.tp)
        else:
            log.warning(f"Cannot open Queen position: Last King position {last_king_pos.ticket} has no TP.")
    else:
        log.warning(f"Cannot open Queen position: No King side positions found.")

# --- Main Loop ---
def run():
    """The main execution loop of the bot."""
    global initial_equity, loop_counter, grid_states, next_magic_number
    mt5_login()
    if ENABLE_TRADING_HOURS:
        log.info(f"TRADING HOURS ENABLED: {TRADING_START_TIME_STR} - {TRADING_END_TIME_STR} UTC")

    # --- Robust Restart Logic ---
    existing_positions = mt5.positions_get(symbol=SYMBOL) or []
    if existing_positions:
        magics = sorted(list(set(p.magic for p in existing_positions)))
        log.info(f"↻ Restarted. Found existing positions with magic numbers: {magics}")
        if magics: next_magic_number = max(magics) + 1
        for i, magic in enumerate(magics):
            grid_name = chr(ord('A') + i % 26)
            pos_in_grid = [p for p in existing_positions if p.magic == magic]
            buys = [p for p in pos_in_grid if p.type == mt5.ORDER_TYPE_BUY]
            sells = [p for p in pos_in_grid if p.type == mt5.ORDER_TYPE_SELL]

            init_buy = next((p for p in buys if ACTION_INITIAL_HEDGE_BUY in p.comment), min(buys, key=lambda p: p.time, default=None))
            init_sell = next((p for p in sells if ACTION_INITIAL_HEDGE_SELL in p.comment), min(sells, key=lambda p: p.time, default=None))

            grid_states[magic] = {
                "name": grid_name,
                "buy_sequence_index": len([p for p in buys if SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]) - 1,
                "sell_sequence_index": len([p for p in sells if SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]) - 1,
                "prev_buy_count": len(buys), "prev_sell_count": len(sells),
                "capped_buy": any(SEARCH_KEYWORD_CAPPED_BUY_SPECIFIC in p.comment for p in pos_in_grid),
                "capped_sell": any(SEARCH_KEYWORD_CAPPED_SELL_SPECIFIC in p.comment for p in pos_in_grid),
                "king_queen_side": "BUY" if any(SEARCH_KEYWORD_KINGSIDE in p.comment for p in buys) else ("SELL" if any(SEARCH_KEYWORD_KINGSIDE in p.comment for p in sells) else None),
                "buy_grid_levels": generate_grid_levels(init_buy.price_open, "BUY") if init_buy else [],
                "sell_grid_levels": generate_grid_levels(init_sell.price_open, "SELL") if init_sell else [],
                "buy_levels_placed": 0, "sell_levels_placed": 0
            }
            log.info(f"Grid {grid_name} (magic {magic}) reconstructed.")
    else:
        hedge_if_empty()

    try:
        while True:
            loop_counter += 1
            if not check_connection():
                log.critical("Connection lost. Waiting 30s before retry.")
                time.sleep(30); continue

            if loop_counter % LOG_BALANCE_INTERVAL == 0:
                if acc := mt5.account_info(): log.info(f"Account Status - Balance: {acc.balance:.2f}, Equity: {acc.equity:.2f}")

            if acc := mt5.account_info():
                if PROFIT_TARGET_AMT > 0 and (acc.equity - initial_equity) >= PROFIT_TARGET_AMT:
                    log.warning(f"🎯 PROFIT TARGET of {PROFIT_TARGET_AMT:.2f} REACHED!")
                    cancel_pending_orders_by_side("BUY"); cancel_pending_orders_by_side("SELL")
                    close_all_symbol_positions(ACTION_SUFFIX_PROFIT_TARGET_CLOSE)
                    grid_states = {}; initial_equity = mt5.account_info().equity
                    log.info(f"Bot reset. New initial equity: {initial_equity:.2f}"); hedge_if_empty()
                    continue
                if MAX_LOSS_AMT > 0 and (initial_equity - acc.equity) >= MAX_LOSS_AMT:
                    log.critical(f"☠️ MAXIMUM LOSS LIMIT of {MAX_LOSS_AMT:.2f} REACHED! BOT STOPPING.")
                    cancel_pending_orders_by_side("BUY"); cancel_pending_orders_by_side("SELL")
                    close_all_symbol_positions(ACTION_SUFFIX_MAX_LOSS_CLOSE)
                    sys.exit(1)

            for magic_key in list(grid_states.keys()):
                if magic_key not in grid_states: continue
                handle_closed_hedge(magic_key)
                if magic_key not in grid_states: continue

                grid_state = grid_states[magic_key]
                if grid_state.get('king_queen_side'):
                    manage_king_queen_sides(magic_key)
                    sync_queen_sl_with_king_tp(magic_key)

                manage_pending_orders(magic_key)
                sync_all_tps("BUY", magic_key); sync_all_tps("SELL", magic_key)
                handle_king_queen_rename(magic_key)

            hedge_if_empty()
            time.sleep(LOOP_MS / 1000.0)

    except KeyboardInterrupt:
        log.info("User requested shutdown (KeyboardInterrupt).")
    except Exception as e:
        log.error(f"An unexpected error occurred in the main loop: {e}\n{traceback.format_exc()}")
    finally:
        log.info("Shutting down MetaTrader 5 connection..."); mt5.shutdown(); log.info("Bot stopped.")

if __name__ == '__main__':
    log.info(f"Starting King/Queen Grid Bot (v121.3-PowerLogic)")
    log.info(f"SYMBOL: {SYMBOL}, LOT_SMALL: {LOT_SMALL}, MAX_LOT: {LOT_MAX}, POWER: {LOT_POWER_MULTIPLIER}")
    log.info(f"GRID_PIPS: {GRID_STEP_PIPS}, PROFIT_PIPS: {PROFIT_PIPS}")
    log.info(f"MAX_POS: {MAX_POSITIONS}, PROFIT_TARGET: {PROFIT_TARGET_AMT}, MAX_LOSS: {MAX_LOSS_AMT if MAX_LOSS_AMT > 0 else 'Disabled'}")
    log.info(f"K/Q TRIGGER: {KING_QUEEN_TRIGGER_COUNT}, MAX PENDING: {MAX_PENDING_ORDERS_AT_ONCE}")
    log.info(f"Trading Hours: Enabled={ENABLE_TRADING_HOURS}, Start='{TRADING_START_TIME_STR}', End='{TRADING_END_TIME_STR}' (UTC)")
    run()